# Preconditioners in NGSolve

Preconditioners are approximate inverses which are used within iterative methods to solve linear or non-linear equations.

Here are some built-in preconditioners in NGSolve:
* Jacobi and Block-Jacobi
* Direct solvers, i.e. sparse factorization
* Multigrid with different block-smoothers
* p-version element-level BDDC

This tutorial quickly introduces how to use these within a solver. (In later tutorials, we will see how to measure condition numbers and customize preconditioners.)

In [ ]:
from netgen.csg import unit_cube
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
%gui tk
import matplotlib.pyplot as plt

### A simple test problem 

We define a simple Poisson problem solver with the name of a preconditioner as argument in order to experiment with preconditioners.


In [ ]:
def SolveProblem(h=0.5, p=1, levels=1, 
                 condense = False,
                 precond="local"):
    """
    Solve Poisson problem on l refinement levels.
        h: coarse mesh size
        p: polynomial degree 
        l: number of refinement levels
        precond: name of a built-in preconditioner
        condense: if true, perform static condensations
    OUTPUT: 
        List of tuples of ndofs and iterations
    """
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    # mesh = Mesh(unit_cube.GenerateMesh(maxh=h))
    fes = H1(mesh, order=p, dirichlet="bottom|left")
    
    u, v = fes.TnT() 
    a = BilinearForm(fes, eliminate_internal=condense)
    a += SymbolicBFI(grad(u)*(grad(v)))
    f = LinearForm(fes)
    f += SymbolicLFI(1*v)
    gfu = GridFunction(fes)
    Draw (gfu)
    c = Preconditioner(a, precond) # 'Register' c to a BEFORE assembly

    steps = []
    
    for l in range(levels):
        if l > 0: mesh.Refine()
        fes.Update()
        a.Assemble()
        f.Assemble()
        gfu.Update()

        # Conjugate gradient solver
        inv = CGSolver(a.mat, c.mat, maxsteps=1000)

        # Solve steps depend on condense 
        if condense:
            f.vec.data += a.harmonic_extension_trans * f.vec
        gfu.vec.data = inv * f.vec
        if condense:
            gfu.vec.data += a.harmonic_extension * gfu.vec
            gfu.vec.data += a.inner_solve * f.vec
        steps.append ( (fes.ndof, inv.GetSteps()) )
        Redraw ()
    return steps

The `Preconditioner` registers itself to the `BilinearForm`. Whenever the `BilinearForm` is re-assembled, the `Preconditioner` is updated as well.

### The `local` preconditioner 

The `local` preconditioner is a simple Jacobi preconditioner. 
The number of CG-iterations with the local preconditioner is proportional to $h^{-1} \sim 2^l$:


In [ ]:
SolveProblem(precond="local")

In [ ]:
res_local = SolveProblem(levels=8, precond="local")
res_local

### Multigrid preconditioner 

A geometric multigrid `Preconditioner` uses the sequence of refined meshes to define a preconditioner of optimal iteration number (and complexity as well). It uses a direct solve on the coarsest level, and block Gauss-Seidel smoothers on the refined levels

In [ ]:
res_mg = SolveProblem(levels=8, precond="multigrid")
res_mg

In [ ]:
plt.xscale("log")
plt.yscale("log")
plt.plot(*zip(*res_local), "-*")
plt.plot(*zip(*res_mg), "-+")
plt.legend(['local', 'mg'])
plt.show()

###  Multigrid implementation for higher order spaces 

For high order elements we use hierarchical spaces, where the (small) sub-spaces $V_E$, $V_F$, and $V_C$ are generated by basis functions associated with edges, faces, and cells:

$$
V_{hp} = V_{p=1} + \sum_{\text{edges }E} V_E + \sum_{\text{faces }F} V_F + \sum_{\text{cells }C} V_C
$$

The system matrix is stored as

$$
A = \left( \begin{array}{cccc}
A_{VV} & A_{VE} & A_{VF} & A_{VC} \\
A_{EV} & A_{EE} & A_{EF} & A_{EC} \\
A_{FV} & A_{FE} & A_{FF} & A_{FC} \\
A_{CV} & A_{CE} & A_{CF} & A_{CC} \\
\end{array} \right)
$$

The $A_{VV}$-block is exactly the system matrix of a lowest order method. 

NGSolve's *multigrid implementation for a high order method uses h-version multigrid for the lowest order block,* and  local block-smoothing for the high order bubble basis functions.

In [ ]:
for p in range(1,10):
    r = SolveProblem(h=0.5, p=p, levels=4, condense=False, 
                     precond="multigrid")
    print ("p =",p,", res =",r)

We observe that the number of iterations grows mildly with the order, and remains bounded with mesh refinement.

Performing static condensation improves the situation:

In [ ]:
for p in range(1,10):
    r = SolveProblem(h=0.5, p=p, levels=4, condense=True, 
                     precond="multigrid")
    print ("p =",p,", res =",r)

### Element-wise BDDC preconditioner

A built-in element-wise BDDC (**B**alancing **D**omain **D**ecomposition preconditioner with **C**onstraints) preconditioner is also available. In contrast to local or multigrid preconditioners, the BDDC preconditioner needs access to the element matrices. This is exactly the reason why we register the preconditioner with the `BilinerForm`, and call the `bfa.Assemble()` later.

In [ ]:
for p in range(1,10):
    r = SolveProblem(h=0.25, p=p, levels=3, condense=True, 
                     precond="bddc")
    print ("p =",p,", res =",r)

The BDDC preconditioner needs more iterations, but the work per iteration is less, so performance is similar to multigrid. **This element-wise BDDC preconditioner works well in shared memory parallel as well as in distributed memory mode.** Go to $\S$[2.1.3](unit-2.1.3-bddc/bddc.ipynb) for more about BDDC preconditioner. 